In [1]:
# result: address, holdings, previousTradeTimestamp, boughtTotal, holdingCurrently

import pandas as pd
import alchemyHelper
import graphqlHelper
import json

# GLOBAL VARIABLES
uniswapV2Router = "0x7a250d5630b4cf539739df2c5dacb4c659f2488d"
uniswapV3Router = "0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45"
weiDivisible = 1000000000000000000

# getting count
with open("mainCOUNT.txt", "r") as f:
    rowNum = int(f.read())
count = rowNum + 1

walletsList = pd.read_csv("wallets_test.csv")
walletsList = walletsList.drop_duplicates()

tokensTraded = 0
stillHoldingOrDusted = 0
sellingIncrementally = 0
accountTransferORNULL = 0
stablecoinTransfer = 0
profits = {}

df = pd.DataFrame(columns=list(['Wallet_Address','Tokens_Traded', 'Still_Holding_OR_Dusted', 'Sold_Incrementally_(NotJet)', 'Transfer_To_Other_Account_OR_NULL', 'Transfer_Stablecoin', 'Average_Profit']))

for i, row in walletsList[rowNum:].iterrows():
    bigDICT = {}
    wallet = row["Address"]
    #print(f"CURRENT WALLET: {wallet}")

    # BUY SECTION
    buyList = alchemyHelper.getBuys(wallet)
    for dict in buyList:
        txnHash = dict["hash"]
        contract = dict["rawContract"]["address"]

        if contract not in bigDICT:
            bigDICT[contract] = {
                "buys": [],
                "sells": []
            }

        bigDICT[contract]['buys'].append(txnHash)

    # SELL SECTION
    sellList = alchemyHelper.getSells(wallet)
    for dict in sellList:
        txnHash = dict["hash"]
        contract = dict["rawContract"]["address"]

        if contract not in bigDICT:
            bigDICT[contract] = {
                "buys": [],
                "sells": []
            }

        bigDICT[contract]['sells'].append(txnHash)

    # bigDICT[contract]
    # ITERATING BIG DICTIONARY
    for contract in bigDICT:
        # REMOVING DUPLICATES
        bigDICT[contract]["buys"] = [*set(bigDICT[contract]["buys"])]
        bigDICT[contract]["sells"] = [*set(bigDICT[contract]["sells"])]

        # CASE0 - IRREGULARITY
        if (bigDICT[contract]["buys"] == []):
            continue

        # CASE1 - still holding token or rugged/dusted
        if (bigDICT[contract]["sells"] == []):
            stillHoldingOrDusted += 1
            continue

        # CASE2 - Great buy and selling incrementally, smart trader
        buyLen = len(bigDICT[contract]["buys"])
        sellLen = len(bigDICT[contract]["sells"])
        diffLen = sellLen / float(buyLen)
        if (diffLen > 2):
            sellingIncrementally += 1

        excludeThisContract = False
        buys = 0.0
        # CHECKING BUYS GRAPHQL
        for buy in bigDICT[contract]["buys"]:
            result = graphqlHelper.getData(buy)
            if (result == -1):
                accountTransferORNULL += 1
                excludeThisContract = True
            elif (result == -2):
                stablecoinTransfer += 1
            else:
                buys += result

        sells = 0.0
        # CHECKING SELLS GRAPHQL
        for sell in bigDICT[contract]["sells"]:
            result = graphqlHelper.getData(sell)
            if (result == -1):
                accountTransferORNULL += 1
                excludeThisContract = True
            elif (result == -2):
                stablecoinTransfer += 1
            else:
                sells += result

        profit = sells - buys
        # STABLECOIN OR JUST TRANSFER
        if (profit == 0.0 or excludeThisContract):
            continue

        profit = ((sells - buys) / buys) * 100.0
        profits[contract] = profit
        tokensTraded += 1

    with open("testRICH.json", "w") as file:
        json.dump(profits, file)

    profitsAVG = sum(profits.values())/len(profits)

    df.loc[count] = [wallet,tokensTraded, stillHoldingOrDusted, sellingIncrementally, accountTransferORNULL, stablecoinTransfer, profitsAVG]

    count += 1

df.to_csv('RESULTS.csv')


In [2]:
pd.set_option('display.max_rows', None)
df.sort_values('Still_Holding_OR_Dusted', ascending=True)

,Wallet_Address,Tokens_Traded,Still_Holding_OR_Dusted,Sold_Incrementally_(NotJet),Transfer_To_Other_Account_OR_NULL,Transfer_Stablecoin,Average_Profit
1,0xc5893e9d79eab042f2cff3ab72c83673034b2b17,38,18,0,0,0,11.705994
